In [1]:
#pip install selenium
#pip install pyyaml ua-parser user-agents fake-useragent

0. Web scraping

In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent
import pandas as pd

# Setting up the chrome driver: pretend to be an organic user
options = Options()
ua = UserAgent()
a = ua.random
user_agent = ua.random
options.add_argument(f'user-agent={user_agent}')
    
# The url for scraping
soup = []

url = 'https://tiki.vn/sach-truyen-tieng-viet/c316'
driver = webdriver.Chrome(options=options, executable_path=r'D:\Software\chromedriver_win32 chrome_97\chromedriver.exe')
driver.get(url)
data = driver.page_source
soup_index = BeautifulSoup(data, 'html.parser') # --> Data is stored in soup
soup.append(soup_index)

for index in range(2,101): #Scraping multiple pages of a website
    url = 'https://tiki.vn/sach-truyen-tieng-viet/c316?page={0}'.format(index)
    driver = webdriver.Chrome(options=options, executable_path=r'D:\Software\chromedriver_win32 chrome_97\chromedriver.exe')
    driver.get(url)
    data = driver.page_source
    soup_index = BeautifulSoup(data, 'html.parser') # --> Data is stored in soup
    soup.append(soup_index)

C:\Users\MINH\AppData\Local\Temp/ipykernel_44652/1286307452.py:18: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=options, executable_path=r'D:\Software\chromedriver_win32 chrome_97\chromedriver.exe')
C:\Users\MINH\AppData\Local\Temp/ipykernel_44652/1286307452.py:26: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=options, executable_path=r'D:\Software\chromedriver_win32 chrome_97\chromedriver.exe')


In [3]:
soup[0]

<html class="no-js" lang="vi"><head><meta charset="utf-8"/><meta content="ie=edge" http-equiv="x-ua-compatible"/><meta content="no-referrer-when-downgrade" name="referrer"/><meta content="7GHar_W5H0cxQxHnW2L1Y6cEIgumXq2ohefKmZXOzT8" name="google-site-verification"/><link href="https://salt.tikicdn.com/media/upload/2018/12/03/0054c1410e38f1b9f3609c244974dd9c.png" rel="icon" sizes="32x32" type="image/png"/><link href="https://salt.tikicdn.com/media/upload/2018/12/03/8bd2f9756646e32ebb076b2f6b050380.png" rel="icon" sizes="16x16" type="image/png"/><link href="https://connect.facebook.net" rel="preconnect"/><link href="https://www.google-analytics.com" rel="preconnect"/><link href="https://fonts.googleapis.com" rel="preconnect"/><link href="https://fonts.gstatic.com" rel="preconnect"/><link href="https://salt.tikicdn.com" rel="preconnect"/><link href="https://frontend.tikicdn.com/_desktop-next" rel="preconnect"/><link href="https://api.tiki.vn/" rel="dns-prefetch"/><link href="https://track

1. Body of the dataframe

In [4]:
# name
product_name_list = []
# price
product_price_list = []
# price
product_discount_list = []
# price
product_rating_list = []
# price
product_sold_list = []

2. Get the data for the dataframe

In [5]:
# 1. Get the product name
print('Product name: ' + soup[0].select('a[class*="product-item"]')[1].select('div[class*="name"]')[0].text.strip())

# 2. Get the product price
print('Product price: ' + soup[0].select('a[class*="product-item"]')[1].select('div[class*="price-discount__price"]')[0].text)

# 3. Get the product discount
print('Product discount: ' + soup[0].select('a[class*="product-item"]')[1].select('div[class*="price-discount__discount"]')[0].text)

# 4. Get the product rating
rating_str = str(soup[0].select('a[class*="product-item"]')[1].select('div[class*="average"]')[0])
i1 = rating_str.find('style="width:') + 13
i2 = rating_str.find('%') + 1
print('Product rating: ' + rating_str[i1:i2])

# 5. Get the product sold
print('Product sold: ' + soup[0].select('a[class*="product-item"]')[1].select('div[class="styles__StyledQtySold-sc-732h27-2 fCfYNm"]')[0].text[7:])

Product name: Tâm Lý Học - Phác Họa Chân Dung Kẻ Phạm Tội
Product price: 99.500 ₫
Product discount: -31%
Product rating: 96%
Product sold: 1000+


In [6]:
for soup_i in soup:
    for product_dtl in soup_i.select('a[class*="product-item"]'):

        #1. Get the name
        product_name_list.append(product_dtl.select('div[class*="name"]')[0].text.strip())

        #2. Get the price
        if len(product_dtl.select('div[class*="price-discount__price"]')) == 0:
            product_price_list.append(None)
        else:
            product_price_list.append(product_dtl.select('div[class*="price-discount__price"]')[0].text)

        #3. Get the discount
        if len(product_dtl.select('div[class*="price-discount__discount"]')) == 0:
            product_discount_list.append(None)
        else:
            product_discount_list.append(product_dtl.select('div[class*="price-discount__discount"]')[0].text)
        
        #4. Get the rating
        if len(product_dtl.select('div[class*="average"]')) == 0:
            product_rating_list.append(None)
        else:
            rating_str = str(product_dtl.select('div[class*="average"]')[0])
            i1 = rating_str.find('style="width:') + 13
            i2 = rating_str.find('%') + 1
            product_rating_list.append(rating_str[i1:i2])
        
        #5. Get the sold number
        if len(product_dtl.select('div[class="styles__StyledQtySold-sc-732h27-2 fCfYNm"]')) == 0:
            product_sold_list.append(None)
        else:
            product_sold_list.append(product_dtl.select('div[class="styles__StyledQtySold-sc-732h27-2 fCfYNm"]')[0].text[7:])

3. Construct the dataframe

In [7]:
#import pandas as pd
dict_to_df = {'product_name': product_name_list, 'product_price': product_price_list
             , 'product_discount': product_discount_list, 'product_rating': product_rating_list
             , 'product_sold': product_sold_list}
df = pd.DataFrame(dict_to_df)
df.head()

,product_name,product_price,product_discount,product_rating,product_sold
0,Cây Cam Ngọt Của Tôi,81.000 ₫,-25%,100%,1000+
1,Tâm Lý Học - Phác Họa Chân Dung Kẻ Phạm Tội,99.500 ₫,-31%,96%,1000+
2,Luật Tâm Thức - Giải Mã Ma Trận Vũ Trụ,208.200 ₫,-26%,94%,1000+
3,"Thiên Tài Bên Trái, Kẻ Điên Bên Phải (Tái Bản)",125.500 ₫,-30%,96%,1000+
4,Here U Are - Tập 1,77.000 ₫,-22%,90%,1000+


In [8]:
for i in df.index:
    
    # Change product_price type:
    if df['product_price'][i] == None:
        df['product_price'][i] = 0
    else:
        df['product_price'][i] = df['product_price'][i].replace('.', '').replace(' ₫', '')
     
    # Change product_discount type:
    if df['product_discount'][i] == None:
        df['product_discount'][i] = 0
    else:
        df['product_discount'][i] = df['product_discount'][i].replace('-', '').replace('%', '')
    
    # Change product_rating type:
    if df['product_rating'][i] == None:
        df['product_rating'][i] = 0
    else:
        df['product_rating'][i] = df['product_rating'][i].replace(' ', '').replace('%', '')
    
    # Change product_sold type:
    if df['product_sold'][i] == None:
        df['product_sold'][i] = 0
    else:
        df['product_sold'][i] = df['product_sold'][i].replace('+', '')
    
df['product_price'] = pd.to_numeric(df['product_price'])
df['product_discount'] = pd.to_numeric(df['product_discount'])
df['product_rating'] = pd.to_numeric(df['product_rating'])
df['product_sold'] = pd.to_numeric(df['product_sold'])
df.head()

,product_name,product_price,product_discount,product_rating,product_sold
0,Cây Cam Ngọt Của Tôi,81000,25,100,1000
1,Tâm Lý Học - Phác Họa Chân Dung Kẻ Phạm Tội,99500,31,96,1000
2,Luật Tâm Thức - Giải Mã Ma Trận Vũ Trụ,208200,26,94,1000
3,"Thiên Tài Bên Trái, Kẻ Điên Bên Phải (Tái Bản)",125500,30,96,1000
4,Here U Are - Tập 1,77000,22,90,1000


In [9]:
df

,product_name,product_price,product_discount,product_rating,product_sold
0,Cây Cam Ngọt Của Tôi,81000,25,100,1000
1,Tâm Lý Học - Phác Họa Chân Dung Kẻ Phạm Tội,99500,31,96,1000
2,Luật Tâm Thức - Giải Mã Ma Trận Vũ Trụ,208200,26,94,1000
3,"Thiên Tài Bên Trái, Kẻ Điên Bên Phải (Tái Bản)",125500,30,96,1000
4,Here U Are - Tập 1,77000,22,90,1000
...,...,...,...,...,...
4795,Nhà Đầu Tư Thông Minh (Tái Bản 2020),143280,28,96,1000
4796,Thế Giới Otome Game Thật Khắc Nghiệt Với Nhân ...,135500,24,94,1000
4797,Ba Người Thầy Vĩ Đại (Tái Bản),65600,31,96,1000
4798,Bộ sách Làm Giàu Từ Chứng Khoán (How To Make M...,700000,0,96,1000


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4800 entries, 0 to 4799
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   product_name      4800 non-null   object
 1   product_price     4800 non-null   int64 
 2   product_discount  4800 non-null   int64 
 3   product_rating    4800 non-null   int64 
 4   product_sold      4800 non-null   int64 
dtypes: int64(4), object(1)
memory usage: 187.6+ KB


In [11]:
df.describe()

,product_price,product_discount,product_rating,product_sold
count,4.800000e+03,4800.000000,4800.000000,4800.000000
mean,1.192890e+05,24.495625,96.402083,737.165208
std,1.151609e+05,11.382401,4.368234,353.765460
min,7.000000e+03,0.000000,0.000000,0.000000
25%,6.250000e+04,23.000000,96.000000,394.000000
50%,8.690000e+04,28.000000,96.000000,1000.000000
75%,1.375000e+05,30.000000,100.000000,1000.000000
max,2.553000e+06,85.000000,100.000000,1000.000000


4. Push the data to SQL Server

In [14]:
import pyodbc

# creating a new db and a new table
new_db_name = "Tiki_VietnameseBooks"
new_table_name = "Tiki_VietnameseBooks"
connection_string = "Driver=SQL Server;Server=DESKTOP-CF5UU1U\SQLEXPRESS;Database={0};Trusted_Connection=Yes;" 
                        # you can also swap Trusted_Connection for UID={your username};PWD={your password}
cnxn = pyodbc.connect(connection_string.format("master"), autocommit=True)
cnxn.cursor().execute('''IF EXISTS(SELECT * FROM sys.databases WHERE [name] = '{0}') DROP DATABASE {0}'''.format(new_db_name))
cnxn.cursor().execute("CREATE DATABASE " + new_db_name)
cnxn.cursor().execute("DROP TABLE IF EXISTS {0}".format(new_table_name))
cnxn.cursor().execute('''USE {1} CREATE TABLE {0}(
                                product_name nvarchar(500), product_price int
                                , product_discount int, product_rating int
                                , product_sold int)'''.format(new_table_name, new_db_name))
for index, row in df.iterrows():
    cnxn.cursor().execute('''INSERT INTO dbo.{0}(
                                product_name, product_price
                                , product_discount, product_rating
                                , product_sold) values (?, ?, ?, ?, ?)'''.format(new_table_name)\
                          , row['product_name'], row['product_price'], row['product_discount']\
                          , row['product_rating'], row['product_sold'])
    cnxn.commit()
cnxn.close()

print("Database and table created")

Database and table created


5. Export dataframe to .csv, .xlsx

In [15]:
#df.to_csv(path, index = False, encoding='utf-8')
#df.to_excel(path, index = False)

path = 'D:\#Tự học DA\Project\Tiki_VietnameseBooks\Tiki_VietnameseBooks.xlsx'

import os
if os.path.exists(path):
    os.remove(path)

df.to_excel(path, index = False)